# 5. 오차역전파법

신경망의 가중치 매개변수의 기울기는 수치 미분을 사용해 구했음.

수치 미분은 단순하고 구현하기도 쉽지만 계산 시간이 오래 걸린다. 이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 **오차역전파법** 에 대해 다뤄볼 것.

오차역전파법을 제대로 이해하는 방법은 두 가지가 있을 것이다.

    1. 수식을 통한 것
    
    2. 계산 그래프를 통한 것
    
여기서는 계산 그래프를 사용해서 **시각적으로 이해**할 것임.

## 계산 그래프

계산 그래프는 계산 과정을 그래프로 나타낸 것.

여기에서는 그래프는 우리가 잘 아는 그래프 자료구조로, **복수의 노드의 에자로 표현**할 것임.

그래서 계산 그래프에 친숙해지기 위한 간단한 문제를 풀어보려 한다. 마지막에는 오차역전파법에 도달할 예정.

### 계산 그래프를 풀다

간단한 문제를 계산 그래프를 사용해서 풀어보자.

    문제 1. 현빈 군은 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세가 10% 부과됩니다.
    
계산 그래프는 계산 과정을 노드와 화살표로 표현한다. 노드는 원으로 표기하고 원 안에 연산 내용을 적습니다.

또, 계산 결과를 화살표 위에 적어 각 노드의 계산 결과가 왼쪽에서 오른쪽으로 전해지게 된다.

<font color = blue> [149페이지 그림 5-1 참고] </font>

처음에 사과의 100원이 "X2" 노드로 흐르고, 200원이 되어 다음 노드로 전달된다. 이제 200원이 "X1.1" 노드를 거쳐 220원이 된다. 따라서 이 계산 그래프에 따르면 최종 답은 220원이 된다.

"X2"와 "X1.1"을 각각 하나의 연산으로 취급해 원 안에 표기했지만, **곱셈만 'X'만을 연산으로 생각** 할 수도 있다. 이렇게 하면 '2'와 '1.1'은 **각각 사과의 개수와 소비세 변수가 되어** 원 밖에 표기하게 된다.

<font color = blue> [150페이지 그림 5-2 참고] </font>

    문제 2. 현빈 군은 슈퍼에서 사과를 2개, 귤 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비세 10%일 때 지불 금액을 구하세요.
    
<font color = blue> [150페이지 그림 5-3 참고] </font>

이 문제에는 덧셈 노드인 "+"가 새로 등장하여 사과와 귤의 금액을 합산한다.

계산 그래프는 왼쪽에서 오른쪽으로 계산을 진행한다. 회로에 전류가 흐르듯 계산 결과가 왼쪽에서 오른쪽으로 전달된다고 생각하면 됨.

따라서 [그림 5-3]에서의 답은 715원이다.

이와 같이 계산 그래프를 이용한 문제풀이는 다음과 같이 진행된다.

    1. 계산 그래프를 구성한다.
    
    2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행된다.
    
여기에서 계산을 왼쪽에서 오른쪽으로 진행하는 단계를 **순전파** 라고 한다. 순전파는 계산 그래프의 출발점부터 종착점으로의 전파이다.

그렇다면 **오른쪽에서 왼쪽의 전파**는 뭐라 할까? 이것을 **역전파**라고 한다.

### 국소적 계산

계산 그래프의 특징은 **'국소적 계산'을 전파함**으로써 최종 결과를 얻는다는 점에서 있음.

국소적이란 '자신과 직점 관계된 작은 범위'라는 뜻. 결국 **전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력**할 수 있다는 것.

구체적인 예를 들어 설명하면, 가령 슈퍼마켓에서 사과 2개를 포함한 여러 식품을 구입하는 경우를 생각해보자.

이는 다음과 같은 계산 그래프로 나타낼 수 있음.

<font color = blue> [150페이지 그림 5-4 참고] </font>

[그림 5-4]에서는 여러 식품을 구입하여 총금액이 4,000원이 되었음. 여기에서 핵심은 각 노드에서의 걔산은 국소적 계산이라는 것.

가령 사과와 그 외의 물품 값을 더하는 계산은 4,000이라는 숫자가 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하기만 하면 됨.

각 노드는 **자신과 관련된 계산(이 예에서는 입력된 숫자의 덧셈)** 외에는 아무것도 신경 쓸 것이 없다.

이처럼 계산 그래프는 **국소적 계산에 집중** 한다. 전체 계산이 제아무리 복잡하더라도 각 단계에서 하는 일은 **해당 노드의 '국소적 계산'** 이다.

국소적인 계산은 단순하지만, 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 해낼 수 있다.

<font color = blue> *NOTE. 자동차 조립은 일반적으로 '조립 라인 작업'에 의한 분업으로 행해진다. 각 담당자(담당 기계)는 단순화된 일만 수행하며 그 일의 결과가 다음 담당자로 전달되어 최종적으로 차를 완성한다. 계산 그래프도 복잡한 계산을 '단순하고 국소적 계산'으로 분할하고 조립 라인 작업을 수행하며 계산 결과를 다음 노드로 전달한다. 복잡한 계산도 분해하면 단순한 계산으로 구성된다는 점은 자동차 조립과 마찬가지이다.* </font>

### 왜 계산 그래프로 푸는가?

계산 그래프의 이점 중 하나는 방금 설명한 **'국소적 계산'**. 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화할 수 있음.

또 다른 이점으로, 계산 그래프는 **중간 계산 결과를 모두 보관**할 수 있음. 예를 들어 사과 2개까지 계산했을 때의 금액은 200원, 소비세를 더하기 전의 금액은 650원인 식이다.

또한, 실제 계산 그래프를 사용하는 큰 이유는 **역전파를 통해 미분을 효율적으로 계산할 수 있는 점**에 있음.

계산 그래프의 역전파를 설명하기 위해 문제 1을 다시 꺼내보자. 문제 1은 사과를 2개 사서 소비세를 포함한 최종 금액을 구하는 것이다.

여기서 가령 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지를 알고 싶다고 하자. 이는 '사과 가격에 대한 지불 금액의 미분'을 구하는 문제에 해당.

기호로 나타낸다면 사과 값을 $x$, 지불 금액은 $L$이라 했을 때, $\frac{\partial L}{\partial x}$ 을 구하는 것.

이 미분 값은 사과 값이 '아주 조금' 올랐을 때 지불 금액이 얼마나 증가하느냐를 표시할 것이다

'사과 가격에 대한 지불 금액의 미분' 같은 값은 계산 그래프에서 역전파를 하면 구할 수 있다.

<font color = blue> [153페이지 그림 5-5 참고] </font>

역전파는 순전파와는 반대 방향의 화살표의 화살표(굵은 선)로 그린다.

이 전파는 국소적 미분을 전달하고 그 미분 값은 화살표의 아래에 적는다. 이 예에서는 역전파는 오른쪽에서 왼쪽으로 '1 -> 1.1 -> 2.2' 순으로 미분 값을 전달한다.

이 결과로부터 **'사과 가격에 대한 지불 금액의 미분'** 의 값은 2.2라 할 수 있다. **사과가 1원 오르면 최종 금액은 2.2원 오른다는 뜻(정확히는 사과 값이 아주 조금 오르면 최종 금액은 그 아주 작은 값의 2.2배만큼 오른다** 라는 뜻.

여기에서는 사과 가격에 대한 미분만을 구했지만, '소비세에 대한 지불 금액의 미분'이나 '사과 개수에 대한 지불 금액의 미분'도 같은 순서로 구할 수 있다.

그리고 그때는 중간까지 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산할 수 있음.

## 연쇄법칙

그동안 해온 계산 그래프의 순전파는 계산 결과를 왼쪽에서 오른쪽으로 전달했음.

그렇지만 역전파는 국소적인 미분을 순방향과는 반대인 오른쪽에서 왼쪽으로 전달한다. 이 국소적 미분을 전달하는 원리는 연쇄법칙에 ㄸ라ㅡㄴ 것.

### 계산 그래프의 역전파

$y = f(x)$라는 계산의 역전파를 그림으로 그려보자.

<font color = blue> [154페이지 그림 5-6 참고] </font>

역전파의 계산 절차는 신호 $E$에 노드의 **국소적 미분** ($\frac{\partial y}{\partial x}$)을 곱한 후, 다음 노드로 전달하는 것.

여기에서 말하는 국소적 미분은 순전파 때의 $y = f(x)$ 계산의 미분을 구한다는 것이며, 이는 x에 대한 y의 미분을 구한다는 것.

가령, $y = f(x) = x^2$ 이라면 $\frac{\partial y}{\partial x} = 2x$가 된다.

이 국소적인 미분을 **상류에서 전달된 값(이 예에서는 E)** 에 곱해 앞쪽 노드로 전달하는 것이다.

이것이 역전파의 계산 순서인데, 이러한 방식을 따르면 목표로 하는 미분 값을 효율적으로 구할 수 있다는 것이 이 전파의 핵심이다.

### 연쇄법칙이란?

연쇄법칙을 설명하려면 우선 합성 함수 이야기부터 시작해야 한다. 합성 함수란 **여러 함수로 구성된 함수**.

예를 들어 $z = {(x+y)}^2$ 라는 식은 다음과 같이 두 개의 식으로 구성된다.

$$z = t^2 \\
t = x + y $$

연쇄법칙은 **합성 함수의 미분에 대한 성질**이며, 다음과 같이 정의된다.

<font color = red> *합성 함수의 미분은 **합성 함수를 구성하는 각 함수의 미분의 곱** 으로 나타낼 수 있다!*
    
위의 식을 예로 설명하면, $\frac{\partial z}{\partial x}$ ($x$에 대한 $z$의 미분)은 $\frac{\partial z}{\partial t}$ ($t$에 대한 $z$의 미분)과 $\frac{\partial t}{\partial x}$ ($x$에 대한 $t$의 미분)의 곱으로 나타낼 수 있다.
    
$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}$$
    
따라서 $\frac{\partial z}{\partial t}$가 사라져, $\frac{\partial z}{\partial x} = \frac{\partial z}{\partial x}$이 성립된다.

그렇다면 연쇄법칙을 써서 위 $z = {(x+y)}^2$의 미분 값인 $\frac{\partial z}{\partial x}$을 구해보자.

$$\frac{\partial z}{\partial t} = 2t \\
\frac{\partial t}{\partial x} = 1 \\
\frac{\partial z}{\partial x} = 2t = 2(x+y)$$

### 연쇄법칙과 계산 그래프

그렇다면 이 계산을 계산 그래프로 나타내보자. 

<font color = blue> [155페이지 그림 5-7 참고] </font>

이 그림과 같이 계산 그래프의 역전파는 오른쪽에 왼쪽으로 신호를 전파한다. 역전파의 계산 절차에서는 노드로 들어온 입력 신호에 **그 노드의 국소적 미분**을 곱한 뒤 다음 노드로 전달한다.

예를 들어 '2제곱' 노드에서의 역전파를 보자. 입력은 $\frac{\partial z}{\partial z}$ 이며, 이에 국소적 미분인 $\frac{\partial z}{\partial t}$ (순전파 시에는 입력이 $t$이고, 출력이 $z$이므로, 이 노드에서 국소적 미분은 다음과 같다)을 곱하고 다음으로 넘긴다.

그림에서 주목할 것은 맨 왼쪽 역전파이다. 이 계산은 결국 'x에 대한 z의 미분'이 되는데, **역전파가 하는 일은 연쇄 법칙의 원리와 같다는 것** 이다.

## 역전파

앞 절에서는 계산 그래프의 역전파가 연쇄법칙에 따라 진행되는 모습을 설명했다. 이번 절에서는 "+"와 "X" 등의 연산을 예로 들어 역전파의 구조를 설명한다.

### 덧셈 노드의 역전파

$z = x+y$ 식을 대상으로 역전파를 살펴보자.

먼저, $z = x+y$의 미분은 다음과 같이 해석적으로 계산할 수 있다.

$$\frac{\partial z}{\partial x} = 1 \\
\frac{\partial z}{\partial y} = 1 $$

위 식에서와 같이 이 식들의 값은 모두 1이 된다. 이를 계산 그래프로는 [그림 5-9]처럼 나타낼 수 있다.

<font color = blue> [157페이지 그림 5-9 참고] </font>

역전파 때는 상류에서 전해진 미분(이 예에서는 $\frac{\partial L}{\partial z}$에 1을 곱하여 하류로 흘린다.

즉, **덧셈 노드의 역전파는 1을 곱하기만 할 뿐이므로 입력된 값을 그대로 다음 노드로 보내게 된다**.

이 예에서는 상류에서 전해진 미분 값을 $\frac{\partial L}{\partial z}$이라고 했는데, 이는 최종적으로 **$L$이라는 값을 출력하는 큰 계산 그래프를 가정하기 때문** 이다. $z = x+y$ 계산은 그 큰 계산 그래프의 중간 어딘가에 존재하고, 상류로부터 $\frac{\partial L}{\partial z}$ 값이 전해진 것이다.

그리고 다시 하류로는 $\frac{\partial L}{\partial x}$과 $\frac{\partial L}{\partial y}$값을 전달하는 것이다.

구체적인 예를 한 번 생각해보자.

가령 '10 + 5 = 15' 라는 계산이 있고, 상류에서 1.3이라는 값이 흘러온다고 하자.

덧셈 노드 역전파는 **입력 신호를 다음 노드로 출력**할 뿐이므로, [그림 5-11]처럼 1.3을 그대로 다음 노드로 전달한다.

<font color = blue> [158페이지 그림 5-11 참고] </font>

### 곱셈 노드의 역전파

$z = xy$라는 식을 생각해보자. 이 식의 미분은 다음과 같다.

$$\frac{\partial z}{\partial x} = y \\
\frac{\partial z}{\partial y} = x$$

따라서 **곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호를 서로 바꾼 값을 곱해서 하류로 보낸다**.
서로 바꾼 값이란 **순전파 때 $x$였다면 역전파에서는 $y$, 순전파 때 $y$였다면 역전파에서는 $x$로 바뀐다** 라는 이야기이다.

마찬가지로 구체적인 예를 생각해보자.

가령 '10 X 5 = 50'이라는 계산이 있고, 역전파 때 상류에서 1.3 값이 흘러온다고 하면, [그림 5-13]처럼 된다.

<font color = blue> [159페이지 그림 5-13 참고] </font>

곱셈의 역전파에서는 **입력 신호를 바꾼 값을 곱하여** 하나는 1.3 X 5 = 6.5, 다른 하나는 1.3 X 10 = 13이 된다.

덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값이 필요하지 않았지만, **곱셈의 역전파는 순방향 입력 신호의 값이 필요**하다. **따라서, 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장**하는 것이 필요하다.

### 사과 쇼핑의 예

이 문제에서는 **사과의 가격, 사과의 개수, 소비세**라는 세 변수 각각이 **최종 금액에 어떻게 영향을 주느냐**를 풀고자 한다.

이는 '사과 가격에 대한 지불 금액의 미분', '사과 개수에 대한 지불 금액의 미분', '소비세에 대한 지불 금액의 미분'을 구하는 것에 해당된다.

<font color = blue> [160페이지 그림 5-14 참고] </font>

결과를 보면 사과 가격의 미분은 2.2, 사과 개수의 미분은 110, 소비세의 미분은 200이다.

이는 소비세와 사과 가격이 같은 양만큼 오르면, 최종 금액에는 소비세가 200의 크기로, 사과 가격이 2.2 크기로 영향을 준다고 해석할 수 잇다.

## 단순한 계층 구현하기

이번 절에서는 지금까지 보아온 '사과 쇼핑' 예를 파이썬을 구현한다.

여기에서는 계산 그래프의 곱셈 노드를 'MultiLayer", 덧셈 노드를 "AddLayer"라는 이름으로 구현한다.

<font color = blue> *NOTE. 다음 절에서는 신경망을 구성하는 '계층' 각각을 하나의 클래스로 구현한다. 여기서 말하는 '계층'이란 신경망의 기능단위이다. 예를 들어 시그모이드 함수를 위한 Sigmoid, 행렬 곱을 위한 Affine 등의 기능을 계층 단위로 구현한다. 그래서 이번 절에서도 곱셈 노드와 덧셈 노드를 '계층' 단위로 구현한다.*

### 곱셈 계층

모든 계층은 forward()와 backward()라는 공통의 메서드(인터페이스)를 갖도록 구현.

    forward()은 순전파, backward()는 역전파를 처리한다.

In [20]:
class MultiLayer:
    def __init__(self): # 앞에서 저장해 둔 변수들을 인스턴스를 호출할 때 초기화해야되기 때문에 __init__이라는 초기화 함수에 초기화값을 넣는다.
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x = x # 인스턴스 변수 self.x에 x값을 저장
        self.y = y # 인스턴스 변수 self.y에 y값을 저장
        out = x*y
        return out
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        return dx, dy

<코드 해석>

    __init__()에서는 인스턴스 변수인 $x$와 $y$를 초기화한다. 이 두 변수는 순전파 시의 입력 값을 유지하기 위해서 사용한다.
    
    forward()에서는 $x$와 $y$를 인수로 받고 두 값을 곱해서 반환한다.
    
    backward()에서는 상류에서 넘어온 미분(dout)에 순전파 떄의 값을 '서로 바꿔' 곱한 후 하류로 흘린다.
    
<font color = purple> None을 넣어주는 기준은 **각자 다른 인스턴스를 부를 때, 각각의 인스턴스가 달라야하는 값들** 이다. 예를 들어, 곱셈 너드에서는 각 층마다 다른 곱셈 노드가 쓰이므로 x와 y가 인스턴스마다 달라야 계산할 수 있다. 따라서 인스턴스를 호출할 때 초기화시키는 __init__에 x와 y를 초기화하여야 한다.
    
이 MultiLayer을 이용하여 앞에서 본 '사과 쇼핑'을 구현해 보자.

<font color = blue> [162페이지 그림 5-16 참고] </font>

#### 클래스에 대한 용어 복습

    class : 개발자가 독자적인 자료형을 만들 수 있게 하는 틀
    
    method : class 안에서 정의된 함수
    
    instance : 클래스를 이용해 다른 변수를 만들 때 그것을 인스턴스라 한다. 즉, 클래스의 객체가 소프트웨어에 실제화할 때를 말함.
    
    인스턴스 변수 : 인스턴스별로 저장되는 변수. self 다음에 속성 이름을 써서 인스턴스 변수를 작성하거나 접근할 수 있음.
    
    __init__ : 클래스를 초기화할 때 쓰는 함수
    
    self : 자신과 자신의 인스턴스를 명시적으로 나타내기 위해 쓰는 것.

In [5]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MultiLayer() # 1번 계층 구현, 사과의 개수와 사과의 값을 곱셈
mul_tax_layer = MultiLayer() # 2번 계층 구현, 1번 계층에서 흘러나온 값에 소비세율을 곱셈

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

int(price)

220

In [10]:
# 역전파

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # 역전파는 순전파와 반대로 인수를 넣어주면 된다.

print(dapple, int(dapple_num), dtax)

2.2 110 200


    backward()가 받는 인수는 '순전파 출력에 대한 미분' 임에 주의하자!
    
    가령 mul_apple_layer라는 곱셈 계층은 순전파 때는 apple_price을 출력하지만, 역전파 때는 apple_price의 미분 값인 dapple_price를 인수로 받는다.

### 덧셈 계층

덧셈 계층을 구현해보자.

In [16]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x+y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

    덧셈 계층에서는 초기화가 필요 없으니 __init__()에서는 아무 일도 하지 않는다. pass는 아무 것도 하지 않으라는 명령이다.
    
곱셈 계층에서는 **순전파 값에 따라 역전파 값이 정해지는 구조**였지만(앞 노드의 $x$와 $y$을 역전파만 해서는 특정할 수 없다)

덧셈 계층에서는 역전파가 단순히 X1, 즉 **값 그대로를 왼쪽으로 보내는 구조**이므로 굳이 인스턴스를 부를 때마다 초기화하지 않아도 된다.

    덧셈 계층의 forward()에서는 입력받은 두 인수 x,y를 더해서 반환한다.
    
    backward()에서는 상류에서 내려온 미분(dout)을 그대로 하류로 흘린다.
    
이상의 덧셈 계층과 곱셈 계층을 사용으로 사과 2개와 귤 3개를 사는 [그림 5-17]의 상황을 구현해보자.

<font color = blue> [164페이지 그림 5-17 참고] </font>

In [26]:
layer_1_1 = MultiLayer()
layer_1_2 = MultiLayer()
layer_2 = AddLayer()
layer_3 = MultiLayer()

apple_num = 2
apple_price = 100
orange_price = 150
orange_num = 3
tax = 1.1

# 순전파
value_1_1 = layer_1_1.forward(apple_num, apple_price)
value_1_2 = layer_1_2.forward(orange_num, orange_price)
value_2 = layer_2.forward(value_1_1, value_1_2)
value_3 = layer_3.forward(value_2, tax)
print(int(value_3))

715


In [33]:
# 역전파
init = 1

dv_1, d_tax = layer_3.backward(init)
dv_2_1, dv_2_2 = layer_2.backward(dv_1)
d_onum, d_oprice = layer_1_2.backward(dv_2_2)
d_anum, d_aprice = layer_1_1.backward(dv_2_1)

print(d_anum, d_aprice, d_oprice, d_onum, d_tax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


    필요한 계층을 만들어 순전파 메서드인 forward()을 적절하게 호출하고,
    
    반대 순서로 역전파 메서드 backward()를 호출하면 원하던 미분이 나온다.

## 활성화 함수 계층 구현하기

계산 그래프를 신경망에 적용해보자.

여기에서는 신경망을 구성하는 층(계층) 각각을 클래스 하나로 구현한다.

우선은 활성화 함수인 ReLu와 Sigmoid 계층을 구현해보자.

<font color = purple> *결국 우리가 원하는 것은 **매개 변수 $W$, $b$에 대해 활성화 함수를 미분** 한 기울기이다. 따라서 활성화 함수에 대한 미분을 알아두어야 한다. </font>

### ReLu 계층

활성화 함수로 사용되는 ReLU의 수식은 다음과 같다.

$$y = \begin{cases} x (x > 0) \\ 0 (x \le 0) \end{cases}$$

이 식에서 **$x$에 대한 $y$의 미분** 은 다음과 같이 구한다.

$$\frac{\partial y}{\partial x} = \begin{cases} 1 (x>0) \\ 0 (x \le 0) \end{cases}$$

순전파 때의 입력인 $x$가 0보다 크면 **역전파는 상류의 값을 그대로 하류로 흘린다** <font color = purple> *미분값이 1이기 때문에* </font>

반면, 순전파 때 $x$가 0 이하면 **역전파 때는 하류로 신호를 보내지 않는다**. 즉, **0을 보낸다**.

<font color = blue> [166페이지 그림 5-18 참고] </font>

이제 이 ReLU 계층을 구현해보자.

    forward()와 backward() 함수는 넘파이 배열을 인수를 받는다고 가정하자.

In [34]:
class Relu:
    def __init__(self):
        self.mask = None 
# 위의 곱셈 노드와 마찬가지로 인스턴스를 호출할 때 앞에서 쌓인 변수들을 초기화해야 하기 때문에 초기화 변수를 넣어준다.
        
    def forward(self, x):
        self.mask = (x <= 0) # 초기화의 대상이다. 한 인스턴스의 호출에서 x<=0이라고 다음 인스턴스도 x<=0일 보장이 없기 때문.
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

    Relu 클래스는 mask라는 인스턴스 변수를 가진다. mask는 True/False로 구성된 넘파이 배열로, 순전파의 입력인 x의 원소 값이 0 이하인 인덱스는 True, 그 외는 False로 유지한다.
    
    예컨대 mask 변수는 아래와 같이 True/False로 구성된 넘파이 배열을 유지한다.

In [36]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [37]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


### Sigmoid 계층

시그모이드 함수는 다음 식을 의미한다.

$$y = \frac{1}{1 + exp(-x)}$$ 이다.

이는 $x, exp, +, /$ 이라는 4개의 노드로 나눌 수 있다. 그림을 참고하자.

<font color = blue> [168페이지 그림 5-19 참고] </font>

[그림 5-19]에는 "X"와 "+" 노드 말고도 "exp"와 "/" 노드가 새롭게 등장한다. 'exp' 노드는 $y=exp(x)$ 계산을 수행하고, '/' 노드는 $y = \frac{1}{x}$ 계산을 수행한다.

그렇다면 여기서는 역전파의 흐름을 오른쪽에서 왼쪽으로 한 단계씩 짚어보자.

#### 1단계

'/' 노드, 즉 $y = \frac{1}{x}$을 미분하면 다음 식이 된다.

$$\begin{align} \frac{\partial y}{\partial x} &= - \frac{1}{x^{2}} \\ & = -y^2 \end{align} $$

이 식에 따르면 역전파 때는 **상류에서 흘러온 값에 $-y^2$을 곱해서 하류로 전달**한다.

<font color = blue> [169페이지 참고] </font>

#### 2단계

'+' 노드는 상류의 값을 여과 없이 하류로 내보낸다.

앞에서의 덧셈 노드와 똑같이 하면 된다.

<font color = blue> [169페이지 참고] </font>

#### 3단계

'exp' 노드는 $y = exp(x)$ 연산을 수행하며, 그 미분은 다음과 같다.

$$\frac{\partial y}{\partial x} = exp(x)$$

계산 그래프에서는 상류의 값에 **순전파 때의 출력을 곱해** 하류로 전파한다.

<font color = blue> [169페이지 참고] </font>

#### 4단계

'X' 노드는 순전파 때의 값을 서로 바꿔 곱한다.

앞에서의 곱셈 노드와 똑같이 하면 된다.

<font color = blue> [169페이지 참고] </font>


### (Back) Sigmoid 계층

이상으로 역전파의 최종 출력인 $\frac{\partial L}{\partial y}y^2exp(-x)$ 값이 하류 노드로 전달된다.

이 최종 값은 **순전파의 입력 $x$와 출력 $y$만으로 계산** 될 수 있다.

그래서 계산 그래프를 모두 묶어 **단순한 'sigmoid' 노드** 하나로 대체할 수 있다.

이렇게 노드를 그룹화하여 Sigmoid 계층의 **세세한 내용을 노출하지 않고 입력과 출력에만 집중할 수 있다**는 것도 중요한 포인트임을 기억할 것.

또한, 이 $\frac{\partial L}{\partial y}y^2exp(-x)$ 을 정리하면 다음과 같다.

$$ \begin{align} \frac{\partial L}{\partial y}y^2exp(-x) & = \frac{\partial L}{\partial y} \frac{1}{(1+exp(-x))^2} exp(-x) \\ &= \frac{\partial L}{\partial y} \frac {1}{1 + exp(-x)} \frac {exp(-x)}{1 + exp(-x)} \\ & = \frac{\partial L}{\partial y}y(1-y) \end{align}$$ 이다.

<font color = red> **이처럼 Sigmoid 계층의 역전파는 순전파의 출력($y$)만으로 계산할 수 있다!** </font>

그렇다면 Sigmoid 계층을 파이썬으로 구현해보자.

In [38]:
class Sigmoid:
    def __init__(self):
        self.out = None # out 또한 인스턴스마다 초기화시켜줘야하니 self.out = None을 설정해준다.
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out # 순전파의 출력값을 지정해주기 위해 self.out을 사용하였다. class 전범위에서 사용하기 위해서이다.
## 뒤 역전파에서 out이라는 변수를 쓰므로 self.out이라 지정해준다.
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

    이 구현에서는 순전파의 출력을 인스턴수 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용한다.

## Affine/Softmax 계층 구현하기

### Affine 계층

신경망의 순전파에서는 가중치 신호의 총합을 계산하기 때문에 행렬의 곱인 **np.dot()을 사용**했다.

예를 들어 다음과 같이 구현했다.

In [41]:
X = np.random.randn(2) # 입력
W = np.random.randn(2,3) # 가중치
B = np.random.randn(3) # 편향

X.shape

(2,)

In [42]:
W.shape

(2, 3)

In [43]:
B.shape

(3,)

In [45]:
Y = np.dot(X, W) + B
print(Y)

[-0.95826614 -0.40184143  0.56831066]


여기에서 $X, W, B$ 각각 형상이 $(2,), (2,3), (3,)$ 인 다차원 배열.

여기서 $Y = XW + B$을 계산하고, 이 $Y$를 활성화 함수로 변환하여 다음 층으로 전파하는 것이 신경망 순전파의 흐름이었다.

**행렬의 곱 계산은 대응하는 차원의 원소 수를 일치** 시키는 것이 핵심.

<font color = blue> *NOTE. 신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서는 **어파인 변환** 이라고 한다. 그래서 여기에서는 어파인 변환을 수행하는 처리를 **'Affine 계층'**이라는 이름으로 구현한다.* </font>

앞에서 수행한 계산을 계산 그래프로 그려보자. 곱을 계산하는 노드를 'dot'이라고 한다면 
    np.dot(X, W) + B 계산은 그림과 같이 구현된다.

<font color = blue> [173페이지 그림 5-24 참고] </font>

그렇다면 이 Affine 계층의 역전파를 생각해보자.

전개해보면 다음과 같은 식이 도출된다.

$$\frac {\partial L}{\partial X} = \frac {\partial L}{\partial Y} \cdot W^{T} \\
\frac {\partial L}{\partial W} = X^{T} \cdot \frac {\partial L}{\partial Y}$$

$X \cdot W + B$는 두 개의 노드인 'dot'과 '+'로 구성되어 있다.

'+' 노드는 앞에서와 본 것과 같이 그대로 하류로 흘려보낸다.

'dot' 노드는 다음과 같이 볼수 있다.

$ Z = X \cdot Y$에서, 우리는 $\frac {\partial L}{\partial X}, \frac {\partial L}{\partial W}$을 구하고자 한다.

따라서 **$X \cdot Y$을 X행렬과 Y행렬에서 편미분한 값** 이 하류로 흘러가야 한다.

$$ \frac {\partial (X\cdot W)}{\partial X} = W^{T} \\
\frac {\partial (X\cdot W)}{\partial W} = X^{T} $$

곱셈 노드와 마찬가지로 상대의 행렬을 곱해줘야 하지만, Transpose해서 곱해준다는 것이 차이점이다.

그러면 이를 바탕으로 계산 그래프의 역전파를 구해보면, [그림 5-25]가 된다.

<font color = blue> [174페이지 그림 5-25 참고] </font>

각 형상에 주의해서 보자.

$X$와 $\frac {\partial L}{\partial X}$은 같은 형상이고, $W$와 $\frac {\partial L}{\partial W}$ 또한 같은 형상이다.

$$\mathrm X = (x_0, x_1, \cdots, x_n) \\
\frac {\partial L}{\partial X} = (\frac {\partial L}{\partial x_0}, \frac {\partial L}{\partial x_1}, \cdots, \frac {\partial L}{\partial x_n})$$

행렬의 곱에서는 대응하는 차원의 원소 수를 일치시켜야 한다, $\frac {\partial L}{\partial Y}$의 형상이 $(3, )$이고, $\frac {\partial L}{\partial X}$의 형상이 $(2, )$가 되는 $\frac {\partial L}{\partial Y} \cdot W^{T}$을 생각한다면, 자연히 식이 유도된다.

$\frac {\partial L}{\partial Y}$이 $(3, )$이고, $W^{T}$의 형상이 $(3,2)$이므로 행렬 계산이 된다.

<font color = purple> 여기서 말하는 $(3, )$은 **3X1 행렬이 아닌 1X3의 벡터임에 주의!** </font>

In [46]:
x_test = np.array([0,1,2]) # 행 벡터를 하나만을 가진 벡터
x_test.shape

(3,)

In [48]:
x_comp = np.array([[0], [1], [2]])
x_comp.shape

(3, 1)

In [51]:
w_test = np.array([[2,0], [2,4], [4,3]])
np.dot(x_test, w_test)

array([10, 10])

### 배치용 Affine 계층

지금까지 설명한 Affine 계층은 입력 데이터로 $X$ 하나만을 고려.

여기에서는 데이터 N개를 묶어 순전파하는 경우를 고려해보자.

<font color = blue> [175페이지 그림 5-27 참고] </font>

여기서 달라진 것은 입력인 $\mathrm X$의 형상이 $(N,2)$가 된 것일 뿐이다. 그 뒤로는 앞에서 배운 계산 그래프댜로 계산이 이루어진다.

다만 편향을 더할 때 주의해야 한다. 순전파 때의 편향 덧셈은 $\mathrm X \cdot \mathrm W$에 대한 편향이 각 데이터에 더해진다. 

In [53]:
X_dot_W = np.array([[0,0,0], [10,10,10]]) # X*W가 2*3 행렬임을 가정. X의 N이 2인 경우
## X가 (2,2) 행렬, W가 (2,3) 행렬이므로 이 곱인 X*W은 2*3 행렬이다.

B = np.array([1,2,3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [54]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

순전파의 편향 덧셈은 각각의 데이터에 더해진다.

그래서 역전파 때는 **각 데이터의 역전파 값이 편향의 원소에 모여야 한다**.

In [55]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [57]:
dB = np.sum(dY, axis = 0)
dB

array([5, 7, 9])

여기서는 데이터가 2개라고 가정하기 때문에, 열이 두 개이다.

편향의 역전파는 **이 두 데이터에 대한 미분을 데이터마다 더해서 구하기** 때문에, np.sum()에서 axis = 0을 설정하여 열의 총합을 구한다.

<font color = purple> 이미지가 2개이면 $Y$ 역시 $(N, 3)$의 (3은 임의의 차원) 형상을 띄게 되므로, $\frac {\partial L}{\partial B} = \frac {\partial L}{\partial Y}$ 또한 $(N, 3)$의 차원을 가지게 된다. 편향은 1차원 데이터이므로 (앞의 층이 아닌 뒤의 층에 의존) 열 총합을 해주는 것이다. </font>

파이썬으로의 구현은 다음과 같다.

In [58]:
class Affine:
    def __init__(self, W, b): # 인스턴스를 호출할 때 초기화하는 변수들을 None으로 설정해주어야 한다.
        ## __init__에서 설정한 변수는 인스턴스를 호출할 때 입력해야 한다. 예를 들어 network = Affine(W_test, b_test) 이런 식으로
        self.W = W
        self.b = b
        self.x = None # 다른 인스턴스를 부를 때 x값이 남으면 안되므로 초기화한다.
        self.dW = None 
        self.db = None # dW와 db도 마찬가지, 똑같은 성질을 가진 노드를 여러 번 사용하기 위해서라고 이해하자.
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # transpose는 .T로 표현할 수 있다.
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        return dx

### Softmax-with-Loss 계층

출력층에서 사용하는 소프트맥스 함수에 대해 설명한다.

이 소프트맥스 함수는 입력 값을 정규화하여 출력한다. 예를 들어 손글씨 숫자 인식에서의 소프트맥스 계층의 출력은 다음과 같다.

[177페이지 그림 5-28 참고]

Softmax 계층은 입력 값을 정규화(**출력의 합이 1이 되도록 변형**)하여 출력한다.

또한, 손글씨 숫자는 가짓수가 10개(10클래스 분류)이므로 Softmax 계층의 입력은 10개가 된다.

<font color = blue> *신경망에서 수행하는 작업은 **학습과 추론 두 가지**이다. 추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다. 예컨대 [그림 5-28]의 신경망을 추론할 때는 마지막 Affine 계층의 출력을 인식 결과로 이용한다. 또한, 신경망에서 정규화하지 않는 출력 결과를 점수(score)라고 한다. 즉, 신경망 추론에서 **답을 하나만 내는 경우에는 가장 높은 점수만 알면 되나 Softmax 계층은 필요 없다**. 반면, 신경망을 학습할 때는 Softmax 계층이 필요하다.* </font>

이제 소프트맥스 계층을 구현해보자. 여기서는 손실 함수인 교차 엔트로피 오차도 포함하여 'Softmax-with-Loss 계층'이라는 이름으로 구현한다.

먼저, Softmax-with-Loss 계층의 계산 그래프를 살펴보자.

<font color = blue> [178페이지 그림 5-29 참고] </font>

<font color = blue> [178페이지 그림 5-30 참고] </font>

[그림 5-30]의 계산 그래프에서 소프트맥스 함수는 'Softmax' 계층으로, 교차 엔트로피 오차는 'Cross Entropy Error' 계층으로 표기한다.

여기에서는 3클래스 분류를 가정하고 이전 계층에서 3개의 입력(점수)를 받는다. 그림과 같이 Softmax 계층은 입력 $(a_1, a_2, a_3)$을 정규화하여 $(y_1, y_2, y_3)$을 출력한다. Cross Entropy Error 계층은 Softmax의 출력 $(y_1, y_2, y_3)$와 정답 레이블 $(t_1, t_2, t_3)$을 받고, 이 데이터로부터 손실 $L$을 출력한다.

[그림 5-30]에서 볼 것은 **역전파의 결과**이다. Softmax 계층의 역전파는 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$의 결과를 내놓고 있다.

$(y_1, y_2, y_3)$은 Softmax 계층의 출력이고, **$(t_1, t_2, t_3)$은 Softmax 계층의 출력과 정답 레이블의 차분** 이다.

신경망의 역전파에서는 <font color = red> **이 차이인 오차가 앞 계층에 전달되는 것** </font>이다. 

신경망 학습의 목적은 신경망의 출력(Softmax의 출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것. 따라서 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달해야 한다. 앞의 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$ 라는 결과는 **신경망의 현재 출력과 정답 레이블의 오차를 있는 그대로** 드러내는 것이다. 

<font color = blue> *NOTE. 우리가 오차제곱합 및 교차 엔트로피 함수를 손실 함수를 사용하는 이유는 역전파가 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$으로 말끔하게 떨어지기 때문이다. 오차제곱합 역시 역전파로 $(y_1 - t_1, y_2 - t_2, y_3 - t_3)$가 말끔하게 떨어진다.* </font>

구체적인 예를 들어보자.

가령 정답 레이블이 $(0,1,0)$일 때 Softmax 계층이 $(0.3, 0.2, 0.5)$을 출력했다고 해보자.

그렇다면 출력에서는 확률이 20%라서, 이 시점의 신경망은 제대로 인식하지 못하고 있다는 것을 알 수 있다. 이 경우 Softmax 계층의 역전파는 $(0.3, -0.8, 0.5)$라는 커다란 오차를 전파한다. **결과적으로 Softmax 계층의 앞 계층들은 그 큰 오차로부터 큰 꼐달음**을 얻게 된다.

이번에는 정답 레이블이 똑같이 $(0,1,0)$일 때, Softmax 계층이 $(0.01, 0.99, 0)$을 출력한 경우이다.

이 경우 Softmax 계층의 역전파가 보내는 오차는 비교적 작은 $(0.01, -0.01, 0)읻. 이번에는 앞 계층으로 전달된 오차가 작으므로 학습하는 정도도 작아진다.

그렇다면 Softmax-with-Loss 계층을 구현해보자.

In [3]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # 신경망에서는 매개변수가 한 데이터에 해당하므로, batch_size를 나눠준다.
        return dx

    softmax()와 cross_entropy_error()을 활용하여 간단하게 구현했다.
    
    역전파 때는 배치의 수인 batch_size로 나눠서 데이터 1개 당 오차를 앞 계층으로 전파한다.

## 오차역전파법 구현하기

앞 절에서 구현한 계층을 조합하면 마치 레고 블록을 조합하듯 신경망을 구축할 수 있다.

이번 절에서는 지금까지 구현한 계층을 조합해서 신경망을 구축할 것이다.

### 신경망 학습의 전체 그림

구체적인 구현에 들어가기 전에, 신경망 학습의 전체 그림을 복습해보자.

#### 전체
    신경망에는 적용 간으한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 학습이라고 한다. 신경망 학습은 다음과 같이 4단계로 수행한다.
    
#### 1단계 - 미니배치
    훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별된 데이터를 미니배치라고 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표이다.
    
#### 2단계 - 기울기 산출
    미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다.
    
#### 3단계 - 매개변수 갱신
    가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다.

#### 4단계 - 반복
    1~3단계를 반복한다.
    
지금까지 설명한 오차역전파법이 등장하는 단계는 두 번째인 '기울기 산출'이다.

앞 장에서는 이 기울기를 구하기 위해서 수치 미분을 사용했다. 그렇지만 수치 미분은 구현하기는 쉽지만 계산이 오래 걸린다.

오차역전파법을 이용하면 느린 수치 미분과 달리 **기울기를 효율적이고 빠르게 구할 수 있다.**

### 오차역전파법을 적용한 신경망 구현하기

본격적인 구현이다. 여기에서는 2층 신경망을 TwoLayerNet 클래스로 구현한다.

In [4]:
import sys, os
os.getcwd()

'/Users/choeunsol/Python'

In [5]:
os.chdir("/Users/choeunsol/deep-learning-from-scratch-master/ch05")
os.getcwd()

'/Users/choeunsol/deep-learning-from-scratch-master/ch05'

In [11]:
sys.path.append(os.pardir)
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
    # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    ## input_size를 행으로, hidden_size을 열로 하는 난수를 생성한다. 나머지 W도 마찬가지로 앞 층 * 뒷 층의 배열을 생성해주자.
        self.params['b1'] = np.zeros(hidden_size)
    ## 편향 b는 뒷 층의 노드 숫자만큼을 생섷한다.
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    # 계층 생성
        self.layers = OrderedDict()
    ## OrderDict()은 순서가 있는 딕셔너리, 즉 딕셔너리에 추가한 순서를 기억
    ## 이 덕분에 각 계층의 forward()만을 호출하면 처리가 완료된다.
        self.layers['Affine1'] = \
        Affine(self.params['W1'], self.params['b1']) # 첫 번째 호춣한 계층
        self.layers['Relu1'] = Relu() # 두 번째 호출한 계층
        self.layers['Affine2'] = \
        Affine(self.params['W2'], self.params['b2']) # 세 번째 호출한 계층
    
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x): 
        for layer in self.layers.values(): # values는 딕셔너리 안에 있는 값들을 호출한다.
            x = layer.forward(x) # layer 안 순전파 함수들을 거치면서 x를 갱신한다.
        
        return x # 순전파의 결과값으로 출력된 x
    
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.LastLayer.forward(y, t) # softmax 함수와 손싫 함수를 동시에 적용
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1) 
        # 이미지 각각에서 가장 높은 값을 가지는 것들을 뽑는다.
        if t.ndim != 1: t = np.argmax(t, axis = 1)
        # 정답 레이블 t가 1차원이 아니라면 t 역시도 똑같이 가장 높은 값을 찾는다.
        # ndim = 1이라는 것은 1차원 벡터라는 것. 1차원 벡터일 경우는 그대로 넣는다. 왜??
    
        accuracy = np.sum(y == t) / float(x.shape[0]) # (여기서는) 이미지의 개수로 나눠주면서 정확도를 측정한다.
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numercial_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
    # 순전파
        self.loss(x,t)
    ## 순전파를 먼저 수행해 softmax와 손실 함수가 적용된 최종 값을 뽑아준다.
    ## loss 함수 안에 predict가 내장되어있으므로 정상적으로 순전파 과정을 수행할 수 있다.
    
    # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
    
        layers = list(self.layers.values())
        layers.reverse()
    ## 역전파이므로 layers에 저장된 value들을 반대로 수행한다. 순전파에서는 그대로.
        for layer in layers:
            dout = layer.backward(dout)
        
    # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
    
        return grads

<코드 해석>

    1. 신경망의 계층을 OrderdDict에 보관하는 점이 중요. OrderedDict은 순서가 있는 딕셔너리로, 순서가 있다는 것은 딕셔너리에 추가한 순서를 기억한다는 것. 그래서 순전파 때는 추가한 순서대로 각 계층의 forward() 메서드를 호출하기만 하면 처리가 완료된다.
    
    2. 마찬가지로 역전파 때는 계층을 반대로 호출하기만 하면 된다. (layers.reverse()가 호출을 반대로 하는 역할을 한다)
    
    3. Affine 계층과 ReLU 계층이 각자의 내부에서 순전파와 역전파를 제대로 처리하고 있으니, 여기에서는 그냥 계층을 올바른 순서로 호출한 뒤 역순으로 호출해주면 끝이다.
    
이처럼 신경망의 구성 요소를 '계층'으로 구분한 덕분에 신경망을 쉽게 구축할 수 있다.

계층을 추가하고 싶다면, 레고 블록을 조립하는 것처럼 **레이어에 계층을 추가**하면 된다.

### 오차역전파법으로 구한 기울기 검증하기

기울기를 구하는 방법은 두 가지 : 첫 번째는 수치 미분, 두 번째는 해석적으로 수식을 풀어 구하는 방법.

여기서는 후자인 해석적 방법이 훨씬 속도가 빠르기 때문에 미분할 때 해석적 방법을 쓸 것이다.

그렇지만 수치 미분이 필요할 때가 있는데, 오차역전파법을 정확히 구현했는지 확인하기 위해 필요하다.

수치 미분의 이점은 구현하기 쉽다는 것. 그래서 종종 **수치 미분의 결과와 오차역전파법의 결과를 비교**하여 오차역전파법을 제댜로 구현했는지 검증하곤 한다. 이것을 **기울기 확인** 이라고 함.

기울기 확인은 다음과 같이 구현한다.

In [14]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = \
load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3] # 행 벡터 인덱싱. 첫 번째 행부터 4번째 행까지 데이터를 뽑는다.
t_batch = t_train[:3]
## 파이썬은 pandas을 이용하지 않으면 열 추출은 반복문을 통해 해야 한다.

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값의 평균을 낸다.

for key in grad_numerical.keys():
    ## keys는 values와 대응되는 함수. 딕셔너리의 이름을 추출한다.
    ## 여기서는 매개변수의 이름들인 'W1', 'W2', 'b1', 'b2'에 해당된다.
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.688070117937395e-10
b1:2.60226807100852e-09
W2:4.714024217955396e-09
b2:1.3972721727428805e-07


이 결과는 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작다고 말해준다.

오차역전파법으로 구한 기울기도 올바름이 드러나면서 실수 없이 구현했다는 믿음이 커지는 것.

### 오차역전파법을 사용한 학습 구현하기 

In [17]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = \
load_mnist(normalize = True, one_hot_label = True)
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

# 하이퍼퍼러미터
## 연구자가 무조건 설정해야 하는 것이므로 익혀놓자.
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

# 1에폭 당 기록
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

# 반복 학습
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기 구하기
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.1543 0.1518
0.90655 0.909
0.9253833333333333 0.9292
0.9353833333333333 0.9374
0.9455 0.9454
0.9518666666666666 0.9481
0.9579166666666666 0.9507
0.9597666666666667 0.9558
0.9632833333333334 0.9583
0.967 0.9603
0.9687666666666667 0.9592
0.97135 0.964
0.9701333333333333 0.9622
0.9744833333333334 0.9659
0.9761166666666666 0.9668
0.9776166666666667 0.9677
0.9770166666666666 0.9677
